In [1]:
import torch

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import DPRContextEncoderTokenizer

In [4]:
model_name = 'facebook/dpr-ctx_encoder-single-nq-base'
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


In [5]:
text = [("How are you?", "I am fine."), ("What's up?", "Not much.")]

In [6]:
tokens_info=context_tokenizer(text, return_tensors='pt', padding=True, \
                              truncation=True, max_length=256)

In [7]:
print(tokens_info)

{'input_ids': tensor([[ 101, 2129, 2024, 2017, 1029,  102, 1045, 2572, 2986, 1012,  102],
        [ 101, 2054, 1005, 1055, 2039, 1029,  102, 2025, 2172, 1012,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [8]:
from transformers import DPRContextEncoder

In [9]:
encoder_model = 'facebook/dpr-ctx_encoder-single-nq-base'
context_encoder = DPRContextEncoder.from_pretrained(encoder_model)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
outputs=context_encoder(**tokens_info)

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [11]:
outputs.pooler_output.shape

torch.Size([2, 768])

In [12]:
def read_and_split_text(filename):
  with open(filename, 'r', encoding='utf-8') as file:
    text = file.read()
  paragraphs = text.split('\n')
  paragraphs = [para.strip() for para in paragraphs if len(para.strip()) > 0]
  return paragraphs

In [14]:
paragraphs = read_and_split_text('/content/drive/MyDrive/companyPolicies.txt')

In [24]:
len(paragraphs)

76

In [25]:
def encode_contexts(text_list):
  embeddings = []
  for text in text_list:
    inputs = context_tokenizer(text, return_tensors='pt', padding=True, \
                               truncation=True, max_length=256)
    outputs = context_encoder(**inputs)
    embeddings.append(outputs.pooler_output)
  return torch.cat(embeddings).detach().numpy()

In [26]:
context_embeddings = encode_contexts(paragraphs)

In [27]:
context_embeddings.shape

(76, 768)

In [28]:
!pip install faiss-cpu --no-cache

In [31]:
import faiss
import numpy as np

In [32]:
import numpy as np
embedding_dim = 768
context_embeddings_np = np.array(context_embeddings).astype('float32')

In [33]:
index = faiss.IndexFlatL2(embedding_dim)
index.add(context_embeddings_np)

In [36]:
from transformers import DPRQuestionEncoder, DPRQuestionEncoderTokenizer

In [35]:
tokenizer_model = 'facebook/dpr-question_encoder-single-nq-base'
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(tokenizer_model)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

In [37]:
encoder_model = 'facebook/dpr-question_encoder-single-nq-base'
question_encoder = DPRQuestionEncoder.from_pretrained(encoder_model)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [38]:
question = 'Drug and Alcohol Policy'
question_inputs = question_tokenizer(question, return_tensors='pt')
question_embedding = question_encoder(**question_inputs).pooler_output.detach().numpy()

In [45]:
D, I = index.search(question_embedding, k=3)

In [40]:
print("D:", D)
print("I:", I)

D: [[72.76531 74.7162  84.38809]]
I: [[48 49 53]]


In [46]:
for i, idx in enumerate(I[0]):
  print(f"{i+1}: {paragraphs[idx]}")
  print(f"distance {D[0][i]}")

1: 6.	Drug and Alcohol Policy
distance 72.76531219482422
2: Policy Objective: The Drug and Alcohol Policy is established to establish clear expectations and guidelines for the responsible use of drugs and alcohol within the organization. This policy aims to maintain a safe, healthy, and productive workplace.
distance 74.71620178222656
3: Testing and Searches: The organization reserves the right to conduct drug and alcohol testing as per applicable laws and regulations. Employees may be subject to testing in cases of reasonable suspicion, post-accident, or as part of routine workplace safety measures.
distance 84.38809204101562


In [48]:
from transformers import BartForConditionalGeneration, BartTokenizer

In [49]:
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [57]:
inputs  = tokenizer(question, return_tensors='pt', max_length=1024, truncation=True)
summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, \
                             length_penalty=2.0, num_beams=4, early_stopping=True)
answer = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [59]:
answer

'Drug and Alcohol Policy. Drug and alcohol policy in the United States. For confidential support call the Samaritans on 08457 90 90 90, visit a local Samaritans branch or see www.samaritans.org.'

In [60]:
def generate_answer_without_context(question):
  inputs  = tokenizer(question, return_tensors='pt', max_length=1024, truncation=True)
  summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, \
                              length_penalty=2.0, num_beams=4, early_stopping=True)
  answer = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
  return answer

In [61]:
question = "what is mobile policy?"
answer = generate_answer_without_context(question)
print(answer)

what is mobile policy?. What is the government's policy on mobile phones? What are its plans for the future of mobile phones. What are the plans for mobile phones in the future?


In [63]:
def generate_answer(contexts):
  input_text = ' '.join(contexts)
  inputs = tokenizer(input_text, return_tensors='pt', max_length=1024, truncation=True)
  summary_ids = model.generate(inputs['input_ids'], max_length=150, min_length=40, \
                               length_penalty=2.0, num_beams=4, early_stopping=True)
  return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [64]:
question = "what is mobile policy?"
# I do not know where the tutor get the down below line function
_,I = search_relevant_contexts(question, question_tokenizer, question_encoder, index, k=5)
top_context = [paragraphs[idx] for idx in I[0]]
answer = generate_answer(top_contexts)

NameError: name 'search_relevant_contexts' is not defined